In [1]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
import pandas as pd 
pd.set_option('display.max_columns', None)

In [2]:
dfDeliveries = pd.read_csv("../data/IPL_Ball_by_Ball_2008_2022.csv") 
dfMatches = pd.read_csv("../data/IPL_Matches_2008_2022.csv")

# Data Cleaning


removing matches where D/L is applied


In [3]:
dfMatches['method'].value_counts()

method
D/L    19
Name: count, dtype: int64

In [4]:
dfMatches['method'].unique()

array([nan, 'D/L'], dtype=object)

In [5]:
dfMatches = dfMatches[dfMatches['method'].isna()]

cleaning season column


In [6]:
dfMatches['Season'] = dfMatches['Season'].replace({'2020/21': '2020', '2009/10': '2010', '2007/08': '2008'})

cleaning team names


In [7]:
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [8]:
dfMatches['Team1']=dfMatches['Team1'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['Team1']=dfMatches['Team1'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['Team1']=dfMatches['Team1'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [9]:
dfMatches['Team2']=dfMatches['Team2'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['Team2']=dfMatches['Team2'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['Team2']=dfMatches['Team2'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [10]:
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

# merged data frame


In [11]:
dfMerged = dfDeliveries.merge(dfMatches, on='ID')

In [12]:
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

Bowling Team Column


In [13]:
dfMerged['BowlingTeam'] = np.where(dfMerged['BattingTeam']==dfMerged['Team1'],
                                       dfMerged['Team2'],
                                       dfMerged['Team1'])

# Batsman Stats


In [14]:
dfBatsmen = dfMerged.groupby('batter')\
            .agg({'ID':'nunique','batsman_run':'sum','isWicketDelivery':'sum'})\
            .reset_index()
dfBatsmen.columns = ['Batter','Innings','Runs','Dismissals']
dfBallsFaced = dfMerged[(dfMerged['extra_type']!='wides')].groupby('batter')['ID'].count().reset_index()
dfBallsFaced.columns = ['Batter','Balls Faced']
dfBatter0s = dfMerged[(dfMerged['batsman_run']==0)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter0s.columns = ['Batter','0s']
dfBatter1s = dfMerged[(dfMerged['batsman_run']==1)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter1s.columns = ['Batter','1s']
dfBatter2s = dfMerged[(dfMerged['batsman_run']==2)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter2s.columns = ['Batter','2s']
dfBatter4s = dfMerged[(dfMerged['batsman_run']==4)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter4s.columns = ['Batter','4s']
dfBatter6s = dfMerged[(dfMerged['batsman_run']==6)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter6s.columns = ['Batter','6s']
dfMatchWise = dfMerged.groupby(['batter','ID'])['batsman_run'].sum().reset_index()
dfHighScore = dfMatchWise.groupby(['batter'])['batsman_run'].max().reset_index()
dfHighScore.columns = ['Batter','High Score']

df25s = dfMatchWise[(dfMatchWise['batsman_run']>=25)&(dfMatchWise['batsman_run']<50)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df25s.columns = ['Batter','25']

df50s = dfMatchWise[(dfMatchWise['batsman_run']>=50)&(dfMatchWise['batsman_run']<100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df50s.columns = ['Batter','50']

df75s = dfMatchWise[(dfMatchWise['batsman_run']>=75)&(dfMatchWise['batsman_run']<100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df75s.columns = ['Batter','75']

df100s = dfMatchWise[(dfMatchWise['batsman_run']>=100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df100s.columns = ['Batter','100']

dfBatsmen = dfBatsmen.merge(dfBallsFaced, on='Batter',how='outer').merge(dfBatter0s, on='Batter',how='outer').merge(dfBatter1s, on='Batter',how='outer').merge(dfBatter2s, on='Batter',how='outer').merge(dfBatter4s, on='Batter',how='outer')\
                     .merge(dfBatter6s, on='Batter',how='outer').merge(dfHighScore, on='Batter',how='outer')\
                     .merge(df25s, on='Batter',how='outer').merge(df50s, on='Batter',how='outer').merge(df75s, on='Batter',how='outer').merge(df100s, on='Batter',how='outer')
dfBatsmen['Strike Rate'] = (dfBatsmen['Runs']/dfBatsmen['Balls Faced'])*100
dfBatsmen['Batting Average'] = dfBatsmen['Runs']/dfBatsmen['Dismissals']
dfBatsmen.fillna(0, inplace=True)
dfBatsmen['notout'] = dfBatsmen['Innings'] - dfBatsmen['Dismissals']
dfBatsmen.sort_values(by='notout', ascending=False).head()

,Batter,Innings,Runs,Dismissals,Balls Faced,0s,1s,2s,4s,6s,High Score,25,50,75,100,Strike Rate,Batting Average,notout
339,MS Dhoni,201,4895,140,3616,1340.0,1490.0,328.0,337.0,226.0,84,65.0,23.0,3.0,0.0,135.370575,34.964286,61
423,RA Jadeja,156,2486,100,1954,732.0,855.0,180.0,182.0,88.0,62,32.0,2.0,0.0,0.0,127.226203,24.860000,56
27,AB de Villiers,167,5168,123,3395,1105.0,1415.0,268.0,412.0,253.0,133,37.0,41.0,11.0,3.0,152.223859,42.016260,44
136,DJ Bravo,110,1560,69,1204,498.0,447.0,111.0,120.0,66.0,70,13.0,5.0,0.0,0.0,129.568106,22.608696,41
597,YK Pathan,149,3143,108,2195,848.0,864.0,153.0,257.0,154.0,100,31.0,14.0,0.0,1.0,143.189066,29.101852,41


In [15]:
dfBatsmen.head(10)

,Batter,Innings,Runs,Dismissals,Balls Faced,0s,1s,2s,4s,6s,High Score,25,50,75,100,Strike Rate,Batting Average,notout
0,A Ashish Reddy,23,280,15,193,61.0,83.0,20.0,16.0,15.0,36,2.0,0.0,0.0,0.0,145.077720,18.666667,8
1,A Badoni,11,161,9,130,57.0,53.0,11.0,11.0,7.0,54,0.0,1.0,0.0,0.0,123.846154,17.888889,2
2,A Chandila,2,4,1,7,3.0,4.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,57.142857,4.000000,1
3,A Chopra,6,53,5,71,45.0,21.0,2.0,7.0,0.0,24,0.0,0.0,0.0,0.0,74.647887,10.600000,1
4,A Choudhary,3,25,2,20,4.0,13.0,1.0,1.0,1.0,15,0.0,0.0,0.0,0.0,125.000000,12.500000,1
5,A Dananjaya,1,4,0,5,2.0,2.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,80.000000,inf,1
6,A Flintoff,3,62,2,53,24.0,23.0,2.0,5.0,2.0,24,0.0,0.0,0.0,0.0,116.981132,31.000000,1
7,A Kumble,15,35,2,47,24.0,21.0,1.0,3.0,0.0,8,0.0,0.0,0.0,0.0,74.468085,17.500000,13
8,A Manohar,7,108,6,75,30.0,30.0,2.0,14.0,3.0,43,2.0,0.0,0.0,0.0,144.000000,18.000000,1
9,A Mishra,54,362,30,389,182.0,172.0,22.0,29.0,5.0,31,3.0,0.0,0.0,0.0,93.059126,12.066667,24


In [16]:
batsman_stats_df = dfBatsmen.rename(columns={'Batter': 'batter', 'Innings': 'matches_played','Runs':'runs_scored','Dismissals':'dismissals','Balls Faced':'balls_faced',
                                             '0s':'0s_scored','1s':'1s_scored','2s':'2s_scored','4s':'4s_scored','6s':'6s_scored','High Score':'high_score','25':'25_scored',
                                             '50':'50_scored','75':'75_scored','100':'100_scored','Strike Rate':'strike_rate','Batting Average':'batting_average'
                                             })

In [17]:
batsman_stats_df.head(15)

,batter,matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate,batting_average,notout
0,A Ashish Reddy,23,280,15,193,61.0,83.0,20.0,16.0,15.0,36,2.0,0.0,0.0,0.0,145.077720,18.666667,8
1,A Badoni,11,161,9,130,57.0,53.0,11.0,11.0,7.0,54,0.0,1.0,0.0,0.0,123.846154,17.888889,2
2,A Chandila,2,4,1,7,3.0,4.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,57.142857,4.000000,1
3,A Chopra,6,53,5,71,45.0,21.0,2.0,7.0,0.0,24,0.0,0.0,0.0,0.0,74.647887,10.600000,1
4,A Choudhary,3,25,2,20,4.0,13.0,1.0,1.0,1.0,15,0.0,0.0,0.0,0.0,125.000000,12.500000,1
5,A Dananjaya,1,4,0,5,2.0,2.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,80.000000,inf,1
6,A Flintoff,3,62,2,53,24.0,23.0,2.0,5.0,2.0,24,0.0,0.0,0.0,0.0,116.981132,31.000000,1
7,A Kumble,15,35,2,47,24.0,21.0,1.0,3.0,0.0,8,0.0,0.0,0.0,0.0,74.468085,17.500000,13
8,A Manohar,7,108,6,75,30.0,30.0,2.0,14.0,3.0,43,2.0,0.0,0.0,0.0,144.000000,18.000000,1
9,A Mishra,54,362,30,389,182.0,172.0,22.0,29.0,5.0,31,3.0,0.0,0.0,0.0,93.059126,12.066667,24


In [18]:
#explosivity rating = Total no. of boundaries hit / Total no. of balls faced
batsman_stats_df['explosivity_rating'] = round((batsman_stats_df['4s_scored']+batsman_stats_df['6s_scored'])/batsman_stats_df['balls_faced'],2)

# Bowling Stats


In [19]:
dfbowler = dfMerged.groupby('bowler')\
            .agg({'ID':'nunique','batsman_run':'sum','extras_run':'sum','isWicketDelivery':'sum'})\
            .reset_index()
dfbowler.columns = ['bowler','Innings','Runs','Extras','Dismissals']

dfBallsBowled = dfMerged[(dfMerged['extra_type']!='wides')].groupby('bowler')['ID'].count().reset_index()
dfBallsBowled.columns = ['bowler','Balls Bowled']

dfbowler4s = dfMerged[(dfMerged['batsman_run']==4)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler4s.columns = ['bowler','4s']
dfbowler0s = dfMerged[(dfMerged['batsman_run']==0)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler0s.columns = ['bowler','0s']
dfbowler1s = dfMerged[(dfMerged['batsman_run']==1)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler1s.columns = ['bowler','1s']
dfbowler2s = dfMerged[(dfMerged['batsman_run']==2)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler2s.columns = ['bowler','2s']

dfbowler6s = dfMerged[(dfMerged['batsman_run']==6)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler6s.columns = ['bowler','6s']
dfMatchWise = dfMerged.groupby(['bowler','ID'])['batsman_run'].sum().reset_index()
dfHighScore = dfMatchWise.groupby(['bowler'])['batsman_run'].max().reset_index()
dfHighScore.columns = ['bowler','Highest Conceeded']

dfbowler = dfbowler.merge(dfBallsBowled, on='bowler',how='outer').merge(dfbowler4s, on='bowler',how='outer')\
                     .merge(dfbowler6s, on='bowler',how='outer').merge(dfbowler0s, on='bowler',how='outer').merge(dfbowler1s, on='bowler',how='outer').merge(dfbowler2s, on='bowler',how='outer').merge(dfHighScore, on='bowler',how='outer')
dfbowler['Strike Rate'] = (dfbowler['Balls Bowled']/dfbowler['Dismissals'])
dfbowler['Bowling Average'] = dfbowler['Runs']/dfbowler['Dismissals']
dfbowler['economy'] = (dfbowler['Runs']/dfbowler['Balls Bowled'])*6
dfbowler.fillna(0, inplace=True)
# Assuming dfbowler is your DataFrame and 'column1' and 'column2' are the columns you want to add
dfbowler['Runs Conceeded'] = dfbowler['Runs'] + dfbowler['Extras']

# If you want to rename the new column, you can use the rename method
new_column_name = 'Runs Conceded'
dfbowler.rename(columns={'Runs Conceeded': new_column_name}, inplace=True)
dfbowler

,bowler,Innings,Runs,Extras,Dismissals,Balls Bowled,4s,6s,0s,1s,2s,Highest Conceeded,Strike Rate,Bowling Average,economy,Runs Conceded
0,A Ashish Reddy,20,386,14,19,264,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,A Badoni,2,11,1,2,12,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,A Chandila,12,242,3,11,234,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,A Choudhary,5,137,7,5,102,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,A Dananjaya,1,46,1,0,24,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,YK Pathan,79,1323,70,45,1105,96.0,55.0,442.0,491.0,53.0,44,24.555556,29.400000,7.183710,1393
468,YS Chahal,128,3465,175,168,2834,202.0,176.0,1120.0,1233.0,169.0,51,16.869048,20.625000,7.335921,3640
469,Yash Dayal,9,283,13,13,195,31.0,12.0,87.0,63.0,12.0,43,15.000000,21.769231,8.707692,296
470,Yuvraj Singh,72,1051,27,39,864,57.0,40.0,282.0,419.0,73.0,35,22.153846,26.948718,7.298611,1078


Need to find the number of single, double or more wickets in a match by bowler.


In [20]:
bowlgroup = dfMerged.groupby(['bowler'])

In [21]:
matches_played_by_bowler = bowlgroup['ID'].unique()

# Create a new dataframe with the bowler and match IDs
matches_played_df = pd.DataFrame({'Bowler': matches_played_by_bowler.index, 'Match IDs': matches_played_by_bowler.values})

matches_played_df

,Bowler,Match IDs
0,A Ashish Reddy,"[980915, 980907, 829773, 829731, 829727, 82971..."
1,A Badoni,"[1304109, 1304083]"
2,A Chandila,"[598052, 598047, 598036, 598032, 598026, 59802..."
3,A Choudhary,"[1082636, 1082633, 1082628, 1082621, 1082591]"
4,A Dananjaya,[1136569]
...,...,...
467,YK Pathan,"[1178394, 1136588, 1082627, 1082604, 1082593, ..."
468,YS Chahal,"[1312200, 1312199, 1312197, 1304114, 1304109, ..."
469,Yash Dayal,"[1312200, 1312197, 1304113, 1304108, 1304103, ..."
470,Yuvraj Singh,"[1136572, 1082630, 1082611, 981003, 980991, 98..."


In [22]:
# Assuming 'isWicketDelivery' column indicates if a wicket was taken
# Adjust the column name if necessary

# Create a list to store the results
wickets_details = []

# Iterate through each row in the matches_played_df dataframe
for index, row in matches_played_df.iterrows():
    bowler = row['Bowler']
    match_ids = row['Match IDs']
    
    # Iterate through each match ID
    for match_id in match_ids:
        # Filter the deliveries dataframe for the specific bowler and match ID
        bowler_match_deliveries = dfMerged[(dfMerged['bowler'] == bowler) & (dfMerged['ID'] == match_id)]
        
        # Count the number of wickets for this bowler in this match
        wickets_count = bowler_match_deliveries['isWicketDelivery'].sum()
        
        # Append the results to the list
        wickets_details.append({
            'Bowler': bowler,
            'Match ID': match_id,
            'Wickets': wickets_count
        })

# Create a DataFrame from the list
wickets_details_df = pd.DataFrame(wickets_details)

# Print the resulting dataframe
wickets_details_df


,Bowler,Match ID,Wickets
0,A Ashish Reddy,980915,1
1,A Ashish Reddy,980907,0
2,A Ashish Reddy,829773,2
3,A Ashish Reddy,829731,1
4,A Ashish Reddy,829727,0
...,...,...,...
11003,Z Khan,335998,1
11004,Z Khan,335996,3
11005,Z Khan,335992,1
11006,Z Khan,335985,4


In [23]:
# Assuming 'Wickets' column indicates the number of wickets taken in a match
# Adjust the column name if necessary

# Group by bowler and Wickets, and count the occurrences
wickets_count_per_bowler = wickets_details_df.groupby(['Bowler', 'Wickets']).size().reset_index(name='Count')

# Pivot the table for better visualization
wickets_count_per_bowler_pivot = wickets_count_per_bowler.pivot(index='Bowler', columns='Wickets', values='Count').fillna(0)

# Print the resulting DataFrame
wickets_count_per_bowler_pivot

Wickets,0,1,2,3,4,5,6
Bowler,,,,,,,
A Ashish Reddy,6.0,10.0,3.0,1.0,0.0,0.0,0.0
A Badoni,0.0,2.0,0.0,0.0,0.0,0.0,0.0
A Chandila,6.0,3.0,2.0,0.0,1.0,0.0,0.0
A Choudhary,1.0,3.0,1.0,0.0,0.0,0.0,0.0
A Dananjaya,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
YK Pathan,49.0,18.0,9.0,3.0,0.0,0.0,0.0
YS Chahal,28.0,52.0,32.0,13.0,2.0,1.0,0.0
Yash Dayal,2.0,3.0,2.0,2.0,0.0,0.0,0.0


merging the two df


In [24]:
# Merge the DataFrames on the 'Bowler' and 'bowler' columns
bowler_merged_df = pd.merge(wickets_count_per_bowler_pivot, dfbowler, left_on='Bowler', right_on='bowler', how='inner')

# Drop the duplicate 'bowler' column (if needed)
# merged_df = merged_df.drop(columns='bowler')

# Print the resulting merged DataFrame
bowler_merged_df

,0,1,2,3,4,5,6,bowler,Innings,Runs,Extras,Dismissals,Balls Bowled,4s,6s,0s,1s,2s,Highest Conceeded,Strike Rate,Bowling Average,economy,Runs Conceded
0,6.0,10.0,3.0,1.0,0.0,0.0,0.0,A Ashish Reddy,20,386,14,19,264,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,A Badoni,2,11,1,2,12,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,6.0,3.0,2.0,0.0,1.0,0.0,0.0,A Chandila,12,242,3,11,234,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,1.0,3.0,1.0,0.0,0.0,0.0,0.0,A Choudhary,5,137,7,5,102,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,A Dananjaya,1,46,1,0,24,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,49.0,18.0,9.0,3.0,0.0,0.0,0.0,YK Pathan,79,1323,70,45,1105,96.0,55.0,442.0,491.0,53.0,44,24.555556,29.400000,7.183710,1393
468,28.0,52.0,32.0,13.0,2.0,1.0,0.0,YS Chahal,128,3465,175,168,2834,202.0,176.0,1120.0,1233.0,169.0,51,16.869048,20.625000,7.335921,3640
469,2.0,3.0,2.0,2.0,0.0,0.0,0.0,Yash Dayal,9,283,13,13,195,31.0,12.0,87.0,63.0,12.0,43,15.000000,21.769231,8.707692,296
470,50.0,12.0,5.0,3.0,2.0,0.0,0.0,Yuvraj Singh,72,1051,27,39,864,57.0,40.0,282.0,419.0,73.0,35,22.153846,26.948718,7.298611,1078


In [25]:
bowler_merged_df.head(5)

,0,1,2,3,4,5,6,bowler,Innings,Runs,Extras,Dismissals,Balls Bowled,4s,6s,0s,1s,2s,Highest Conceeded,Strike Rate,Bowling Average,economy,Runs Conceded
0,6.0,10.0,3.0,1.0,0.0,0.0,0.0,A Ashish Reddy,20,386,14,19,264,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,A Badoni,2,11,1,2,12,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,6.0,3.0,2.0,0.0,1.0,0.0,0.0,A Chandila,12,242,3,11,234,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,1.0,3.0,1.0,0.0,0.0,0.0,0.0,A Choudhary,5,137,7,5,102,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,A Dananjaya,1,46,1,0,24,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47


In [26]:
list(bowler_merged_df.columns)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 'bowler',
 'Innings',
 'Runs',
 'Extras',
 'Dismissals',
 'Balls Bowled',
 '4s',
 '6s',
 '0s',
 '1s',
 '2s',
 'Highest Conceeded',
 'Strike Rate',
 'Bowling Average',
 'economy',
 'Runs Conceded']

In [27]:
bowler_stats_df = bowler_merged_df.rename(columns={'Innings': 'matches_played','Runs':'runs_conceded','Dismissals':'wickets_taken','Balls Bowled':'balls_bowled',
                                             '0s':'0s_conceded','1s':'1s_conceded','2s':'2s_conceded','4s':'4s_conceded','6s':'6s_conceded','Highest Conceeded':'highest_conceded',
                                             'Strike Rate':'strike_rate','Bowling Average':'bowling_average','Runs Conceded':'total_runs_conceded','Extras':'extras_runs_conceded',
                                             0:'0_wickets_taken',1:'1_wickets_taken',2:'2_wickets_taken',3:'3_wickets_taken',4:'4_wickets_taken',5:'5_wickets_taken',
                                             6:'6_wickets_taken'
                                             })

In [28]:
bowler_stats_df.head(5)

,0_wickets_taken,1_wickets_taken,2_wickets_taken,3_wickets_taken,4_wickets_taken,5_wickets_taken,6_wickets_taken,bowler,matches_played,runs_conceded,extras_runs_conceded,wickets_taken,balls_bowled,4s_conceded,6s_conceded,0s_conceded,1s_conceded,2s_conceded,highest_conceded,strike_rate,bowling_average,economy,total_runs_conceded
0,6.0,10.0,3.0,1.0,0.0,0.0,0.0,A Ashish Reddy,20,386,14,19,264,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,A Badoni,2,11,1,2,12,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,6.0,3.0,2.0,0.0,1.0,0.0,0.0,A Chandila,12,242,3,11,234,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,1.0,3.0,1.0,0.0,0.0,0.0,0.0,A Choudhary,5,137,7,5,102,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,A Dananjaya,1,46,1,0,24,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47


In [29]:
batsman_stats_df.rename(columns={'matches_played': 'batter_matches_played'}, inplace=True)

In [30]:
bowler_stats_df.rename(columns={'matches_played': 'bowler_matches_played'}, inplace=True)


In [31]:
list(batsman_stats_df)

['batter',
 'batter_matches_played',
 'runs_scored',
 'dismissals',
 'balls_faced',
 '0s_scored',
 '1s_scored',
 '2s_scored',
 '4s_scored',
 '6s_scored',
 'high_score',
 '25_scored',
 '50_scored',
 '75_scored',
 '100_scored',
 'strike_rate',
 'batting_average',
 'notout',
 'explosivity_rating']

In [32]:
list(bowler_stats_df)

['0_wickets_taken',
 '1_wickets_taken',
 '2_wickets_taken',
 '3_wickets_taken',
 '4_wickets_taken',
 '5_wickets_taken',
 '6_wickets_taken',
 'bowler',
 'bowler_matches_played',
 'runs_conceded',
 'extras_runs_conceded',
 'wickets_taken',
 'balls_bowled',
 '4s_conceded',
 '6s_conceded',
 '0s_conceded',
 '1s_conceded',
 '2s_conceded',
 'highest_conceded',
 'strike_rate',
 'bowling_average',
 'economy',
 'total_runs_conceded']

# player only stats for frontend


In [43]:
batsman_stats_df['batter'].nunique()

605

In [44]:
bowler_stats_df['bowler'].nunique()

472

In [53]:

# Assuming batsman_stats_df and bowler_stats_df are your dataframes
# Merge on different columns ('batter' and 'bowler') and use 'outer' to include all players
player_stats_merged_df = pd.merge(batsman_stats_df, bowler_stats_df, left_on='batter', right_on='bowler', how='outer')

# 'outer' includes all players, even if they only have stats in one of the dataframes
# NaN values will be present for players without stats in one of the dataframes


In [54]:
player_stats_merged_df.head(10)

,batter,batter_matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate_x,batting_average,notout,explosivity_rating,0_wickets_taken,1_wickets_taken,2_wickets_taken,3_wickets_taken,4_wickets_taken,5_wickets_taken,6_wickets_taken,bowler,bowler_matches_played,runs_conceded,extras_runs_conceded,wickets_taken,balls_bowled,4s_conceded,6s_conceded,0s_conceded,1s_conceded,2s_conceded,highest_conceded,strike_rate_y,bowling_average,economy,total_runs_conceded
0,A Ashish Reddy,23.0,280.0,15.0,193.0,61.0,83.0,20.0,16.0,15.0,36.0,2.0,0.0,0.0,0.0,145.077720,18.666667,8.0,0.16,6.0,10.0,3.0,1.0,0.0,0.0,0.0,A Ashish Reddy,20.0,386.0,14.0,19.0,264.0,26.0,20.0,89.0,110.0,23.0,36.0,13.894737,20.315789,8.772727,400.0
1,A Badoni,11.0,161.0,9.0,130.0,57.0,53.0,11.0,11.0,7.0,54.0,0.0,1.0,0.0,0.0,123.846154,17.888889,2.0,0.14,0.0,2.0,0.0,0.0,0.0,0.0,0.0,A Badoni,2.0,11.0,1.0,2.0,12.0,0.0,0.0,4.0,5.0,3.0,6.0,6.000000,5.500000,5.500000,12.0
2,A Chandila,2.0,4.0,1.0,7.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,57.142857,4.000000,1.0,0.00,6.0,3.0,2.0,0.0,1.0,0.0,0.0,A Chandila,12.0,242.0,3.0,11.0,234.0,17.0,10.0,105.0,91.0,10.0,39.0,21.272727,22.000000,6.205128,245.0
3,A Chopra,6.0,53.0,5.0,71.0,45.0,21.0,2.0,7.0,0.0,24.0,0.0,0.0,0.0,0.0,74.647887,10.600000,1.0,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A Choudhary,3.0,25.0,2.0,20.0,4.0,13.0,1.0,1.0,1.0,15.0,0.0,0.0,0.0,0.0,125.000000,12.500000,1.0,0.10,1.0,3.0,1.0,0.0,0.0,0.0,0.0,A Choudhary,5.0,137.0,7.0,5.0,102.0,13.0,6.0,49.0,31.0,9.0,51.0,20.400000,27.400000,8.058824,144.0
5,A Dananjaya,1.0,4.0,0.0,5.0,2.0,2.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,80.000000,inf,1.0,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,A Dananjaya,1.0,46.0,1.0,0.0,24.0,2.0,4.0,7.0,10.0,2.0,46.0,inf,inf,11.500000,47.0
6,A Flintoff,3.0,62.0,2.0,53.0,24.0,23.0,2.0,5.0,2.0,24.0,0.0,0.0,0.0,0.0,116.981132,31.000000,1.0,0.13,1.0,2.0,0.0,0.0,0.0,0.0,0.0,A Flintoff,3.0,105.0,1.0,2.0,66.0,7.0,7.0,21.0,27.0,4.0,50.0,33.000000,52.500000,9.545455,106.0
7,A Kumble,15.0,35.0,2.0,47.0,24.0,21.0,1.0,3.0,0.0,8.0,0.0,0.0,0.0,0.0,74.468085,17.500000,13.0,0.06,15.0,16.0,6.0,1.0,2.0,2.0,0.0,A Kumble,42.0,1027.0,62.0,49.0,970.0,72.0,35.0,410.0,406.0,57.0,46.0,19.795918,20.959184,6.352577,1089.0
8,A Manohar,7.0,108.0,6.0,75.0,30.0,30.0,2.0,14.0,3.0,43.0,2.0,0.0,0.0,0.0,144.000000,18.000000,1.0,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A Mishra,54.0,362.0,30.0,389.0,182.0,172.0,22.0,29.0,5.0,31.0,3.0,0.0,0.0,0.0,93.059126,12.066667,24.0,0.09,52.0,52.0,26.0,14.0,5.0,1.0,0.0,A Mishra,150.0,3831.0,125.0,171.0,3212.0,225.0,171.0,1226.0,1387.0,247.0,55.0,18.783626,22.403509,7.156289,3956.0


In [55]:
# Rename 'batter' column to 'player'
player_stats_merged_df.rename(columns={'batter': 'player'}, inplace=True)

# Drop 'bowler' column
player_stats_merged_df.drop(columns='bowler', inplace=True)


In [56]:
player_stats_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   player                 605 non-null    object 
 1   batter_matches_played  605 non-null    float64
 2   runs_scored            605 non-null    float64
 3   dismissals             605 non-null    float64
 4   balls_faced            605 non-null    float64
 5   0s_scored              605 non-null    float64
 6   1s_scored              605 non-null    float64
 7   2s_scored              605 non-null    float64
 8   4s_scored              605 non-null    float64
 9   6s_scored              605 non-null    float64
 10  high_score             605 non-null    float64
 11  25_scored              605 non-null    float64
 12  50_scored              605 non-null    float64
 13  75_scored              605 non-null    float64
 14  100_scored             605 non-null    float64
 15  strike

remove rows for which player column is null


In [57]:
player_stats_merged_df = player_stats_merged_df.dropna(subset=['player'])

In [62]:
player_stats_merged_df.describe()

,batter_matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate_x,batting_average,notout,explosivity_rating,0_wickets_taken,1_wickets_taken,2_wickets_taken,3_wickets_taken,4_wickets_taken,5_wickets_taken,6_wickets_taken,bowler_matches_played,runs_conceded,extras_runs_conceded,wickets_taken,balls_bowled,4s_conceded,6s_conceded,0s_conceded,1s_conceded,2s_conceded,highest_conceded,strike_rate_y,bowling_average,economy,total_runs_conceded
count,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,574.000000,605.000000,605.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,376.000000,376.000000,425.000000,425.000000
mean,23.161983,457.457851,18.175207,356.761983,147.889256,136.876033,23.320661,41.497521,17.307438,36.178512,4.519008,2.323967,0.509091,0.122314,103.536058,15.945374,4.986777,0.121273,10.247059,8.310588,4.614118,1.767059,0.510588,0.117647,0.014118,25.581176,641.995294,34.515294,25.555294,501.482353,58.136471,24.235294,207.844706,192.651765,32.804706,39.145882,22.508029,30.625708,8.492934,676.510588
std,36.687560,968.518851,31.100814,729.898307,290.823299,288.838671,48.825846,91.192098,39.310464,33.887785,9.540696,6.409650,1.705659,0.565200,41.075873,10.938916,7.605755,0.085304,12.691464,11.261158,6.946683,2.996830,1.096954,0.400818,0.118115,32.677146,850.092634,48.342922,36.733710,690.807331,78.658446,31.850642,294.455421,272.019277,46.134565,13.476478,10.045259,15.525837,2.497699,894.898806
min,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,3.000000,15.000000,2.000000,17.000000,9.000000,6.000000,1.000000,1.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,80.645161,7.762500,1.000000,0.070000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,93.000000,4.000000,3.000000,66.000000,7.000000,4.000000,24.000000,25.000000,4.000000,31.000000,17.205128,22.333333,7.335921,97.000000
50%,8.000000,71.000000,6.000000,70.000000,33.000000,23.000000,4.000000,6.000000,2.000000,24.000000,0.000000,0.000000,0.000000,0.000000,110.619469,14.253846,2.000000,0.130000,5.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,12.000000,289.000000,15.000000,10.000000,214.000000,25.000000,12.000000,83.000000,85.000000,14.000000,41.000000,20.111111,26.974359,8.040000,304.000000
75%,23.000000,326.000000,18.000000,274.000000,114.000000,102.000000,18.000000,26.000000,13.000000,55.000000,4.000000,1.000000,0.000000,0.000000,128.989362,22.882184,6.000000,0.170000,14.000000,11.000000,6.000000,2.000000,1.000000,0.000000,0.000000,33.000000,876.000000,46.000000,33.000000,660.000000,75.000000,32.000000,275.000000,247.000000,44.000000,49.000000,24.906250,34.051282,8.839117,929.000000
max,221.000000,6462.000000,203.000000,5049.000000,1910.000000,2159.000000,371.000000,692.000000,336.000000,175.000000,65.000000,53.000000,17.000000,6.000000,400.000000,88.000000,61.000000,1.000000,96.000000,72.000000,43.000000,17.000000,9.000000,3.000000,1.000000,180.000000,4249.000000,292.000000,207.000000,3778.000000,423.000000,176.000000,1551.000000,1750.000000,269.000000,69.000000,85.000000,126.000000,36.000000,4451.000000


replacing inf values with nan


In [59]:
import numpy as np

# Assuming your DataFrame is named df
player_stats_merged_df.loc[:, 'batting_average'] = player_stats_merged_df['batting_average'].replace([np.inf, -np.inf], np.nan)


In [61]:
player_stats_merged_df.loc[:, 'bowling_average'] = player_stats_merged_df['bowling_average'].replace([np.inf, -np.inf], np.nan)
player_stats_merged_df.loc[:, 'strike_rate_y'] = player_stats_merged_df['strike_rate_y'].replace([np.inf, -np.inf], np.nan)

save csv and json


In [63]:
player_stats_merged_df.to_csv('../derived/player_stats_data.csv',index=False)

setting key for json


In [64]:

player_stats_merged_df_with_player_as_key = player_stats_merged_df.set_index('player')
player_stats_merged_df_with_player_as_key.to_json('../derived/player_stats_data.json', orient='index')


In [40]:
# Save the DataFrame to a JSON file
player_stats_merged_df.to_json('../derived/player_stats_data.json', orient='index')


done


# for final stats


In [46]:
list(dfMerged)

['ID',
 'innings',
 'overs',
 'ballnumber',
 'batter',
 'bowler',
 'non-striker',
 'extra_type',
 'batsman_run',
 'extras_run',
 'total_run',
 'non_boundary',
 'isWicketDelivery',
 'player_out',
 'kind',
 'fielders_involved',
 'BattingTeam',
 'City',
 'Date',
 'Season',
 'MatchNumber',
 'Team1',
 'Team2',
 'Venue',
 'TossWinner',
 'TossDecision',
 'SuperOver',
 'WinningTeam',
 'WonBy',
 'Margin',
 'method',
 'Player_of_Match',
 'Team1Players',
 'Team2Players',
 'Umpire1',
 'Umpire2',
 'BowlingTeam']

In [47]:
test_first_df = pd.merge(dfMerged, batsman_stats_df, on='batter', how='left')
test_first_df.head(10)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,BowlingTeam,batter_matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate,batting_average,notout,explosivity_rating
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,79,2736,69,1849,761.0,615.0,130.0,271.0,126.0,124,19.0,14.0,6.0,5.0,147.971877,39.652174,10,0.21
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21
5,1312200,1,0,6,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21
6,1312200,1,1,1,JC Buttler,Yash Dayal,YBK Jaiswal,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,79,2736,69,1849,761.0,615.0,130.0,271.0,126.0,124,19.0,14.0,6.0,5.0,147

In [48]:
# Merge based on the 'bowler' column
final_stats_df = pd.merge(test_first_df, bowler_stats_df, on='bowler', how='left')

# Display the head of the final merged DataFrame
final_stats_df.head(10)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,BowlingTeam,batter_matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate_x,batting_average,notout,explosivity_rating,0_wickets_taken,1_wickets_taken,2_wickets_taken,3_wickets_taken,4_wickets_taken,5_wickets_taken,6_wickets_taken,bowler_matches_played,runs_conceded,extras_runs_conceded,wickets_taken,balls_bowled,4s_conceded,6s_conceded,0s_conceded,1s_conceded,2s_conceded,highest_conceded,strike_rate_y,bowling_average,economy,total_runs_conceded
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,79,2736,69,1849,761.0,615.0,130.0,271.0,126.0,124,19.0,14.0,6.0,5.0,147.971877,39.652174,10,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902
5,1312200,1,0,6,YBK Jaiswal,

In [50]:
len(final_stats_df.columns)

77

In [51]:
# Save the final merged DataFrame to a CSV file
final_stats_df.to_csv('../derived/final_stats_data.csv', index=False)


# Adding current score, target,runs_left, balls_left, wickets_left


In [2]:
df = pd.read_csv("../derived/final_stats_data.csv",low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222746 entries, 0 to 222745
Data columns (total 77 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     222746 non-null  int64  
 1   innings                222746 non-null  int64  
 2   overs                  222746 non-null  int64  
 3   ballnumber             222746 non-null  int64  
 4   batter                 222746 non-null  object 
 5   bowler                 222746 non-null  object 
 6   non-striker            222746 non-null  object 
 7   extra_type             11853 non-null   object 
 8   batsman_run            222746 non-null  int64  
 9   extras_run             222746 non-null  int64  
 10  total_run              222746 non-null  int64  
 11  non_boundary           222746 non-null  int64  
 12  isWicketDelivery       222746 non-null  int64  
 13  player_out             10996 non-null   object 
 14  kind                   10996 non-nul

In [4]:
total_score = df.groupby(['ID', 'innings']).sum()['total_run'].reset_index()

In [10]:
total_score.head()

,ID,innings,total_run,target
0,335982,1,222,223
2,335983,1,240,241
4,335984,1,129,130
6,335985,1,165,166
8,335986,1,110,111


In [8]:
total_score = total_score[total_score['innings']==1]

In [9]:
total_score['target'] = total_score['total_run'] + 1

merge with the df dataframe


In [11]:
all_df = df.merge(total_score[['ID','target']], on='ID')

In [12]:
all_df[['ID','target']]

,ID,target
0,1312200,131
1,1312200,131
2,1312200,131
3,1312200,131
4,1312200,131
...,...,...
222741,335982,223
222742,335982,223
222743,335982,223
222744,335982,223


adding current score


In [14]:
all_df['current_score'] = all_df.groupby(['ID', 'innings'])['total_run'].cumsum()

# Reset the index if needed
all_df = all_df.reset_index(drop=True)

In [19]:
pd.set_option('display.max_rows', None)

In [ ]:
all_df[['ID','innings','overs','ballnumber','total_run','current_score','balls_left','wickets_left','runs_left']].iloc[120:180]

adding balls left and wickets left


In [25]:
all_df['balls_left'] = np.where((120 - all_df['overs']*6 - all_df['ballnumber'] >= 0) & (all_df['innings'] == 1),
                                   120 - all_df['overs']*6 - all_df['ballnumber'],
                                   np.where((120 - all_df['overs']*6 - all_df['ballnumber'] >= 0) & (all_df['innings'] == 2),
                                            120 - all_df['overs']*6 - all_df['ballnumber'],
                                            0))


In [32]:
all_df['wickets_left'] = 10 - all_df.groupby(['ID', 'innings'])['isWicketDelivery'].cumsum()


adding runs left only for second innings, 0 if for first innnig


In [38]:
all_df['runs_left'] = np.where((all_df['innings'] == 2) & (all_df['target'] - all_df['current_score'] >= 0),
                                  all_df['target'] - all_df['current_score'], 0)


In [41]:
all_df.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,BowlingTeam,batter_matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate_x,batting_average,notout,explosivity_rating,0_wickets_taken,1_wickets_taken,2_wickets_taken,3_wickets_taken,4_wickets_taken,5_wickets_taken,6_wickets_taken,bowler_matches_played,runs_conceded,extras_runs_conceded,wickets_taken,balls_bowled,4s_conceded,6s_conceded,0s_conceded,1s_conceded,2s_conceded,highest_conceded,strike_rate_y,bowling_average,economy,total_runs_conceded,target,current_score,balls_left,wickets_left,runs_left
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902,131,0,119,10,0
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902,131,1,118,10,0
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,79,2736,69,1849,761.0,615.0,130.0,271.0,126.0,124,19.0,14.0,6.0,5.0,147.971877,39.652174,10,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902,131,2,117,10,0
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,90,2738,164,115,2014,299.0,103.0,897.0,653.0,120.0,52,17.513043,23.808696,8.156902,2902,131,2,116,10,0
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans,23,547,23,406,177.0,133.0,14.0,62.0,22.0,68,6.0,3.0,0.0,0.0,134.729064,23.782609,0,0.21,21.0,35.0,23.0,10.0,1.0,0.0,0.0,9

In [43]:
len(all_df.columns)

82

In [44]:
len(df.columns)

77

save as csv


In [ ]:
all_df.to_csv('../derived/final_stats_data.csv', index=False)